0.1 Import packages

In [2]:
import requests
import pandas as pd

0.2 Some useful url's for the MediaDive api

In [4]:
#MediaDive urls
comp_url = f'https://mediadive.dsmz.de/rest/medium/' #comp can be a mix of solutions and ingredients
taxa_url = f'https://mediadive.dsmz.de/rest/medium-strains/'
solu_url = f'https://mediadive.dsmz.de/rest/solution/' #returns ingredients
ingr_url = f'https://mediadive.dsmz.de/rest/ingredient/' #simplest form (I think)
    # All followed by a DSMZ media ID 

0.3 Media list to consider (refer to media_compound_dataset for Komodo media rankings)

In [6]:
id_list = ['65',
 '514',
 '830',
 '92',
 '1',
 '535',
 '220',
 '11',
 '693',
 '104',
 '53',
 '645',
 '110',
 '585',
 '58',
 '429',
 '372',
 '545',
 '98',
 '9',
 '63',
 '553',
 '78',
 '84',
 '104b',
 '215',
 '381',
 '425',
 '119',
 '535b',
 '554',
 '736',
 '67',
 '804',
 '27',
 '83',
 '641.1',
 '354',
 '123',
 '621',
 '760.1',
 '339.1',
 '339',
 '595',
 '31',
 '144',
 '330',
 '214',
 '593',
 '371']

1.1 Function to list all components (less useful)

In [8]:
#id_list = ['1', '1001'] #List of DSMZ media IDs

def get_composition_list():
    base_url = 'https://mediadive.dsmz.de/rest/medium/{}'
    composition_data = []

    for id in id_list:
        url = base_url.format(id)
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            # Traverse the nested structure to extract compounds
            solutions = data.get('data', {}).get('solutions', [])
            for solution in solutions:
                recipe = solution.get('recipe', [])
                for item in recipe:
                    if 'compound' in item:  # Check for 'compound' and 'compound_id'
                        composition_data.append({
                            'component': item.get('compound'),
                            'component_id': item.get('compound_id')
                        })
                    elif 'solution' in item:  # Check for 'solution' and 'solution_id'
                        composition_data.append({
                            'component': item.get('solution'),
                            'component_id': item.get('solution_id')
                        })
        else:
            print(f"Request failed with status code: {response.status_code}")

    # Convert the list of dictionaries to a DataFrame
    composition_df = pd.DataFrame(composition_data)
    return composition_df

In [9]:
#comp_list = get_composition_list()
#comp_list

1.2 Function to list a 'media id', its 'components', and the 'component ids'

In [11]:
#id_list = ['1', '1001'] #List of DSMZ media IDs

def get_composition():
    base_url = 'https://mediadive.dsmz.de/rest/medium/{}'
    composition_data = []

    for id in id_list:
        url = base_url.format(id)
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            # Initialize lists to store components and ids for each media
            components = []
            component_ids = []
            # Traverse the nested structure to extract compounds
            solutions = data.get('data', {}).get('solutions', [])
            for solution in solutions:
                recipe = solution.get('recipe', [])
                for item in recipe:
                    if 'compound' in item:  # Check for 'compound' and 'compound_id'
                        components.append(item.get('compound'))
                        component_ids.append(item.get('compound_id'))
                    elif 'solution' in item:  # Check for 'solution' and 'solution_id'
                        components.append(item.get('solution'))
                        component_ids.append(item.get('solution_id'))
            # Append data for this media to composition_data
            composition_data.append({
                'media_id': id,
                'components': components,
                'component_ids': component_ids
            })     
        else:
            print(f"Request failed with status code: {response.status_code}")
    
        print(f'Retrieved data for {id}')

    # Convert the list of dictionaries to a DataFrame
    composition_df = pd.DataFrame(composition_data)
    return composition_df

# Can add something to get the compounds and IDs for the complex solution mixtures, but maybe easier to leave as "Vitamin mixture" etc for now
    # i.e., in the place of compounds some api entries were solutions, but each of those solutions has a list of compounds

In [12]:
media_comps = get_composition()
media_comps

Retrieved data for 65
Retrieved data for 514
Retrieved data for 830
Retrieved data for 92
Retrieved data for 1
Retrieved data for 535
Retrieved data for 220
Retrieved data for 11
Retrieved data for 693
Retrieved data for 104
Retrieved data for 53
Retrieved data for 645
Retrieved data for 110
Retrieved data for 585
Retrieved data for 58
Retrieved data for 429
Retrieved data for 372
Retrieved data for 545
Retrieved data for 98
Retrieved data for 9
Retrieved data for 63
Retrieved data for 553
Retrieved data for 78
Retrieved data for 84
Retrieved data for 104b
Retrieved data for 215
Retrieved data for 381
Retrieved data for 425
Retrieved data for 119
Retrieved data for 535b
Retrieved data for 554
Retrieved data for 736
Retrieved data for 67
Retrieved data for 804
Retrieved data for 27
Retrieved data for 83
Request failed with status code: 404
Retrieved data for 641.1
Retrieved data for 354
Retrieved data for 123
Retrieved data for 621
Request failed with status code: 404
Retrieved data for

,media_id,components,component_ids
0,65,"[Glucose, Yeast extract, Malt extract, CaCO3, ...","[5, 16, 116, 13, 3, 4]"
1,514,"[Bacto peptone, Yeast extract, Fe(III) citrate...","[19, 16, 45, 43, 453, 110, 81, 61, 55, 174, 52..."
2,830,"[Yeast extract, Proteose peptone, Casamino aci...","[16, 208, 101, 5, 199, 113, 10, 8, 3, 4]"
3,92,"[Trypticase soy broth, Yeast extract, Agar, Di...","[99, 16, 3, 4]"
4,1,"[Peptone, Meat extract, Agar, Distilled water]","[1, 2, 3, 4]"
5,535,"[Trypticase soy broth, Agar, Distilled water]","[99, 3, 4]"
6,220,"[Casein peptone, Soy peptone, NaCl, Agar, Dist...","[21, 75, 43, 3, 4]"
7,11,"[Casein peptone, Meat extract, Yeast extract, ...","[21, 2, 16, 5, 22, 10, 23, 24, 8, 14, 4]"
8,693,"[Columbia agar base, Defibrinated sheep blood,...","[2110, 1717, 21, 75, 94, 43, 10, 4, 186, 4, 1,..."
9,104,"[Trypticase peptone, Peptone, Yeast extract, B...","[124, 1, 16, 85, 5, 10, 22, 174, 25, 125, 124,..."


1.3 Function to list the 'strain_id', 'species', 'ccno' (culture collection number), and 'bacdive_id' for associated organisms of a given media

In [14]:
#id_list = ['1','1001']  # List of DSMZ media IDs

def get_strains():
    base_url = 'https://mediadive.dsmz.de/rest/medium-strains/{}'
    strain_data = []

    for id in id_list:
        url = base_url.format(id)
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            
            strains = data.get('data', [])
            for strain in strains:
                strain_data.append({
                    'media_id': id,
                    'strain_id': strain.get('id'),
                    'species': strain.get('species'),
                    'ccno': strain.get('ccno'),
                    'bacdive_id': strain.get('bacdive_id')
                })     
        else:
            print(f"Request failed with status code: {response.status_code}")
    
        print(f'Retrieved data for {id}')

    # Convert the list of dictionaries to a DataFrame
    strain_df = pd.DataFrame(strain_data)
    return strain_df

In [15]:
strains_df = get_strains()
strains_df

Retrieved data for 65
Retrieved data for 514
Retrieved data for 830
Retrieved data for 92
Retrieved data for 1
Retrieved data for 535
Retrieved data for 220
Retrieved data for 11
Retrieved data for 693
Retrieved data for 104
Retrieved data for 53
Retrieved data for 645
Retrieved data for 110
Retrieved data for 585
Retrieved data for 58
Retrieved data for 429
Retrieved data for 372
Retrieved data for 545
Retrieved data for 98
Retrieved data for 9
Retrieved data for 63
Retrieved data for 553
Retrieved data for 78
Retrieved data for 84
Retrieved data for 104b
Retrieved data for 215
Retrieved data for 381
Retrieved data for 425
Retrieved data for 119
Retrieved data for 535b
Retrieved data for 554
Retrieved data for 736
Retrieved data for 67
Retrieved data for 804
Retrieved data for 27
Retrieved data for 83
Request failed with status code: 404
Retrieved data for 641.1
Retrieved data for 354
Retrieved data for 123
Retrieved data for 621
Request failed with status code: 404
Retrieved data for

,media_id,strain_id,species,ccno,bacdive_id
0,65,233,Rhodococcus sp.,DSM 364,10957.0
1,65,359,Pseudonocardia autotrophica,DSM 535,13324.0
2,65,488,Streptomyces tanashiensis,DSM 731,16033.0
3,65,495,Streptomyces clavuligerus,DSM 738,15101.0
4,65,500,Rhodococcus erythropolis,DSM 743,10853.0
...,...,...,...,...,...
28942,371,14347,Natronococcus roseus,DSM 25473,24068.0
28943,371,16447,Natrialba chahannaoensis,DSM 29986,130845.0
28944,371,16448,Natrialba hulunbeirensis,DSM 29987,130844.0
28945,371,29106,Salicibibacter cibarius,DSM 111417,160200.0


1.4 Format and merge our dataframes

In [17]:
# MediaDive enters nan as NaN values, so conventional .dropna doesn't appear to work
strains_df['bacdive_id'] = strains_df['bacdive_id'].astype(str)  # convert to string
substring = 'nan'
filter = strains_df['bacdive_id'].str.contains(substring) # create filter
filtered_df = strains_df[~filter]
filtered_df

,media_id,strain_id,species,ccno,bacdive_id
0,65,233,Rhodococcus sp.,DSM 364,10957.0
1,65,359,Pseudonocardia autotrophica,DSM 535,13324.0
2,65,488,Streptomyces tanashiensis,DSM 731,16033.0
3,65,495,Streptomyces clavuligerus,DSM 738,15101.0
4,65,500,Rhodococcus erythropolis,DSM 743,10853.0
...,...,...,...,...,...
28942,371,14347,Natronococcus roseus,DSM 25473,24068.0
28943,371,16447,Natrialba chahannaoensis,DSM 29986,130845.0
28944,371,16448,Natrialba hulunbeirensis,DSM 29987,130844.0
28945,371,29106,Salicibibacter cibarius,DSM 111417,160200.0


In [18]:
grouped = filtered_df.groupby('media_id')['bacdive_id'].agg(','.join).dropna().reset_index()
merged = media_comps.merge(grouped, on='media_id', how='left')
merged

,media_id,components,component_ids,bacdive_id
0,65,"[Glucose, Yeast extract, Malt extract, CaCO3, ...","[5, 16, 116, 13, 3, 4]","10957.0,13324.0,16033.0,15101.0,10853.0,15620...."
1,514,"[Bacto peptone, Yeast extract, Fe(III) citrate...","[19, 16, 45, 43, 453, 110, 81, 61, 55, 174, 52...","11976.0,17717.0,5591.0,17261.0,17249.0,23365.0..."
2,830,"[Yeast extract, Proteose peptone, Casamino aci...","[16, 208, 101, 5, 199, 113, 10, 8, 3, 4]","2885.0,17580.0,131872.0,132778.0,17584.0,17591..."
3,92,"[Trypticase soy broth, Yeast extract, Agar, Di...","[99, 16, 3, 4]","14445.0,14446.0,4969.0,14447.0,7537.0,12758.0,..."
4,1,"[Peptone, Meat extract, Agar, Distilled water]","[1, 2, 3, 4]","654.0,11477.0,11984.0,748.0,598.0,1172.0,642.0..."
5,535,"[Trypticase soy broth, Agar, Distilled water]","[99, 3, 4]","10851.0,10897.0,13324.0,13355.0,13353.0,4968.0..."
6,220,"[Casein peptone, Soy peptone, NaCl, Agar, Dist...","[21, 75, 43, 3, 4]","683.0,1419.0,11981.0,8069.0,640.0,1448.0,578.0..."
7,11,"[Casein peptone, Meat extract, Yeast extract, ...","[21, 2, 16, 5, 22, 10, 23, 24, 8, 14, 4]","24124.0,24122.0,6650.0,6651.0,6652.0,14390.0,6..."
8,693,"[Columbia agar base, Defibrinated sheep blood,...","[2110, 1717, 21, 75, 94, 43, 10, 4, 186, 4, 1,...","1584.0,1601.0,14753.0,14755.0,1585.0,17256.0,1..."
9,104,"[Trypticase peptone, Peptone, Yeast extract, B...","[124, 1, 16, 85, 5, 10, 22, 174, 25, 125, 124,...","17860.0,17118.0,18068.0,17969.0,5436.0,5448.0,..."


NEXT STEPS: 

(1) Incorporate some type of biological identifier (EC, KO, gene, etc.) into this table...some media have thousands of strains associated with it, so we either need to do a broader reduction of taxa (e.g., group by genus, type strains, etc) or use reactions/ids associated with the media_id. 

(2) Could also do two columns on the same table of taxa-associated ec's and media component-associated ec. I think bacdive may also have enzyme information embedded there, so we can double-check whether there's a way to use the api to grab all of the information from the same database (can cross-reference later). 
Note: also see https://bacdive.dsmz.de/dashboard, some interesting trends are shown.

(3) This is a subset of the 50 most abundantly used media (by taxa) from the Komodo database--can expand our list to include many more media.

(4) Format these outputs for our machine learning model

(5) Data visualization

...also in general, comparing the overlap in these components between different media_id labels...will the ML clustering essentially do this for us? Or are there better ways to visualize?